# Requêtes APIs Orion

Bienvenue dans la documentation de ce notebook de test d'APIs permettant de récupérer des données utiles pour la création d'un score de risque pour les feux de forêt.


## Introduction

Les feux de forêt sont l'un des fléaux les plus dévastateurs pour les écosystèmes et les communautés qui dépendent d'eux. Dans de nombreux cas, la prévention est la meilleure solution, et cela nécessite des outils de prévision précis pour mesurer le risque de déclenchement de feux de forêt.

Ce notebook a pour objectif de tester différentes APIs pour créer un score de risque de feux de forêt en utilisant des informations telles que la météo, la végétation, l'humidité des sols, la topographie, et d'autres données environnementales. En explorant différentes sources de données, nous espérons trouver les meilleures informations pour créer un modèle de prédiction précis qui aidera les pompiers à prendre des mesures pour prévenir les feux de forêt.

Nous avons sélectionné quelques APIs mentionnées dans le papier [Wildfire Danger Prediction and Understanding With Deep Learning](https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2022GL099368) et nous allons explorer chacune d'entre elles en détail. Cette documentation servira de guide pour comprendre comment requêter et utiliser ces APIs.

Nous espérons que ce notebook sera utile pour les personnes qui cherchent à améliorer la prévention des feux de forêt et à protéger nos écosystèmes.


### Comment commencer ?
#### Clonage du repo
Commencer par ouvrir son IDE (Pycharm, VS, terminal...), et cloner le repository Git : 
```
git clone git@github.com:pyronear/pyro-risks.git
```

/!\ Il faut que votre clé SSH soit configurée dans Github. Si ce n'est pas fait, voir [ici](https://docs.github.com/en/authentication/connecting-to-github-with-ssh/adding-a-new-ssh-key-to-your-github-account).

#### Environnement virtuel
Activer son [environnement virtuel](https://realpython.com/python-virtual-environments-a-primer/) :
```bash
source venv/bin/activate
```

/!\ Si vous n'avez pas configuré votre environnement virtuel, rendez vous à la racine du repo, et tapez dans votre terminal : 
```bash
python -m venv venv
pip install -r requirements.txt
```

#### Notebook
Lancez ensuite votre notebook :

```bash
pip install jupyter
jupyter notebook
```

Une fenêtre s'ouvre dans votre navigateur, c'est le notebook !
 
Se rendre dans `pyro_risks/notebooks`, puis vous pouvez créer votre propre notebook, ou bien lancer un notebook déjà créé.

## Requête des APIs mentionnées dans le papier

Le papier mentionne 6 datasets : 

 - **Daily weather data**, de **[ERA-5 Land](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land?tab=overview)** (Muñoz-Sabater et al., 2021) (température maximale à 2 m, vitesse maximale du vent, humidité relative minimale, précipitations totales, température maximale du point de rosée à 2 m et la pression maximale en surface), aggrégées à la journée.

 - **Satellite variables** de **[MODIS](https://www.earthdata.nasa.gov/learn/find-data/near-real-time/firms/mcd14dl-nrt)**, dont Normalized Difference Vegetation Index (NDVI; Didan, 2015), day and night Land Surface Temperature (LST; Wan et al., 2015).

 - **Soil moisture index** de **[European Drought Observatory](https://edo.jrc.ec.europa.eu/edov2/php/index.php?id=1000)** (Cammalleri et al., 2017).

 - **Roads distance, waterway distance, and yearly population density** de **WorldPop** (Tatem, 2017).

 - **Elevation and Slope** de **[Copernicus EU-DEM](https://land.copernicus.eu/imagery-in-situ/eu-dem/eu-dem-v1.1)** (Bashfield & Keim, 2011).

 - **Ten variables with the fraction of classes** (?) de **[Copernicus Corine Land Cover](https://land.copernicus.eu/pan-european/corine-land-cover)** (Büttner, 2014).


### 1. Daily weather data - ERA-5 Land
#### Requête
Documentation [ici](https://confluence.ecmwf.int/display/CKB/ERA5-Land%3A+data+documentation).

In [1]:
# TROUBLESHOOT : Jupyter ne reconnaissait pas mon venv donc j'ai dû réinstaller mes packages ici :
"""
%pip install geopandas
%pip install xarray
%pip install netCDF4
%pip install imblearn
%pip install xgboost
%pip install dvc
%pip install plot_metric
"""

'\n%pip install geopandas\n%pip install xarray\n%pip install netCDF4\n%pip install imblearn\n%pip install xgboost\n%pip install dvc\n%pip install plot_metric\n'

On vérifie que l'on se trouve à la racine du répertoire pour lancer le code. Si ce n'est pas le cas, on se rend à la racine du répertoire :

In [1]:
import os
if os.getcwd().split("/")[-1]=="notebooks":
    os.chdir("../../")

On importe ici le package de l'API Copernicus Climate Data Store (CDS), et nos identifiants personnels pour effectuer des requêtes. Ces identifiants peuvent être créés sur le site [Copernicus Climate Data Store](https://cds.climate.copernicus.eu/user/login?destination=/api/). 

Ensuite, il s'agit de configurer votre PC pour faire des requêtes sans rentrer votre nom d'utilisateur et votre mot de passe dans le code. Pour ce faire, se référer à la [documentation suivante](https://cds.climate.copernicus.eu/api-how-to), ou à la [vidéo suivante](https://www.youtube.com/watch?v=AmF1nn7o6Hc&ab_channel=ClimateUnboxed). 

Ces identifiants vont être récupérés directement dans le code.

In [2]:
import cdsapi
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd
import sys

Téléchargement des données pour une journée : il y a une API, mais on est limités dans la taille du fichier résultant ...

In [3]:
"""c = cdsapi.Client()
year=2022
month=6
#day=['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31']
day=30
time=['00:00','01:00','02:00','03:00','04:00','05:00','06:00','07:00','08:00','09:00','10:00','11:00','12:00','13:00','14:00','15:00','16:00','17:00','18:00','19:00','20:00','21:00','22:00','23:00']
output_path = os.getcwd()+"/pyro_risks/notebooks/datastore"
file_path = os.path.join(output_path, f"era5land_{year}_{month}_{day}.nc")

# Requête des valeurs
c.retrieve(
    "reanalysis-era5-land",
    {
        "variable": [
            "2m_temperature",
            "2m_dewpoint_temperature",
            "surface_pressure",
            "10m_u_component_of_wind",
            "10m_v_component_of_wind",
            "total_precipitation",
        ],
        "year": year,
        "month": month,
        "day": day,
        "time":time,
        "area": [
            51,
            -6,
            41,
            10,
        ],
        "format": "netcdf",
    },
    file_path,
)"""

'c = cdsapi.Client()\nyear=2022\nmonth=6\n#day=[\'01\',\'02\',\'03\',\'04\',\'05\',\'06\',\'07\',\'08\',\'09\',\'10\',\'11\',\'12\',\'13\',\'14\',\'15\',\'16\',\'17\',\'18\',\'19\',\'20\',\'21\',\'22\',\'23\',\'24\',\'25\',\'26\',\'27\',\'28\',\'29\',\'30\',\'31\']\nday=30\ntime=[\'00:00\',\'01:00\',\'02:00\',\'03:00\',\'04:00\',\'05:00\',\'06:00\',\'07:00\',\'08:00\',\'09:00\',\'10:00\',\'11:00\',\'12:00\',\'13:00\',\'14:00\',\'15:00\',\'16:00\',\'17:00\',\'18:00\',\'19:00\',\'20:00\',\'21:00\',\'22:00\',\'23:00\']\noutput_path = os.getcwd()+"/pyro_risks/notebooks/datastore"\nfile_path = os.path.join(output_path, f"era5land_{year}_{month}_{day}.nc")\n\n# Requête des valeurs\nc.retrieve(\n    "reanalysis-era5-land",\n    {\n        "variable": [\n            "2m_temperature",\n            "2m_dewpoint_temperature",\n            "surface_pressure",\n            "10m_u_component_of_wind",\n            "10m_v_component_of_wind",\n            "total_precipitation",\n        ],\n        "ye

On tente de faire une boucle pour requêter 1 mois :

In [5]:
"""c = cdsapi.Client()
years=['2009', '2010','2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022'] 
months=['01','02','03','04','05','06','07','08','09','10','11','12']
days=['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31']
time=['00:00','01:00','02:00','03:00','04:00','05:00','06:00','07:00','08:00','09:00','10:00','11:00','12:00','13:00','14:00','15:00','16:00','17:00','18:00','19:00','20:00','21:00','22:00','23:00']

# Requête des valeurs
for year in years:
    # Check path existe et le crée  s'il n'existe pas
    output_path = os.getcwd()+"/pyro_risks/notebooks/datastore/ERA5Land/"+str(year)
    os.makedirs(output_path, mode=0o777, exist_ok=False)
    for month in months:
        for day in days:
            try:
                file_path = os.path.join(output_path, f"era5land_{year}_{month}_{day}.nc")
                c.retrieve(
                    "reanalysis-era5-land",
                    {
                        "variable": [
                            "2m_temperature",
                            "2m_dewpoint_temperature",
                            "surface_pressure",
                            "10m_u_component_of_wind",
                            "10m_v_component_of_wind",
                            "total_precipitation",
                        ],
                        "year": year,
                        "month": month,
                        "day": day,
                        "time":time,
                        "area": [
                            51,
                            -6,
                            41,
                            10,
                        ],
                        "format": "netcdf",
                    },
                    file_path,
                )
            except:
                print("Unexpected error:", sys.exc_info()[0])
"""

2023-03-15 06:06:03,143 INFO Welcome to the CDS
2023-03-15 06:06:03,146 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land
2023-03-15 06:06:03,423 INFO Request is completed
2023-03-15 06:06:03,425 INFO Downloading https://download-0007-clone.copernicus-climate.eu/cache-compute-0007/cache/data4/adaptor.mars.internal-1678729651.3429487-23103-14-9c0fdc08-5726-4fc2-a2b2-d750755b99b1.nc to /Users/camillemodeste/Documents/Workflow/Pyronear/pyro-risks/pyro_risks/notebooks/datastore/ERA5Land/2011/era5land_2011_01_01.nc (4.5M)
2023-03-15 06:06:04,235 INFO Download rate 5.5M/s                                                                                                   
2023-03-15 06:06:04,323 INFO Welcome to the CDS
2023-03-15 06:06:04,325 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land
2023-03-15 06:06:04,489 INFO Request is queued
2023-03-15 06:06:05,543 INFO Request is running
2023-03-15 06:06:37

2023-03-15 06:09:29,075 INFO Download rate 7.9M/s                                                                                                   
2023-03-15 06:09:29,173 INFO Welcome to the CDS
2023-03-15 06:09:29,174 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land
2023-03-15 06:09:29,375 INFO Request is queued
2023-03-15 06:09:30,431 INFO Request is running
2023-03-15 06:10:01,945 INFO Request is completed
2023-03-15 06:10:01,948 INFO Downloading https://download-0018.copernicus-climate.eu/cache-compute-0018/cache/data3/adaptor.mars.internal-1678856991.7335362-4410-11-f8245aaf-91f0-4cd0-94e0-1cf2a0dfe567.nc to /Users/camillemodeste/Documents/Workflow/Pyronear/pyro-risks/pyro_risks/notebooks/datastore/ERA5Land/2011/era5land_2011_01_12.nc (4.5M)
2023-03-15 06:10:03,229 INFO Download rate 3.5M/s                                                                                                   
2023-03-15 06:10:03,321 INFO Welcome to the C

2023-03-15 06:11:19,468 INFO Download rate 3.8M/s                                                                                                   
2023-03-15 06:11:19,549 INFO Welcome to the CDS
2023-03-15 06:11:19,550 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land
2023-03-15 06:11:19,699 INFO Downloading https://download-0002-clone.copernicus-climate.eu/cache-compute-0002/cache/data5/adaptor.mars.internal-1678730417.8967369-15731-20-753d4575-2734-49ae-a90d-754cde690331.nc to /Users/camillemodeste/Documents/Workflow/Pyronear/pyro-risks/pyro_risks/notebooks/datastore/ERA5Land/2011/era5land_2011_01_24.nc (4.5M)
2023-03-15 06:11:20,265 INFO Download rate 7.9M/s                                                                                                   
2023-03-15 06:11:20,346 INFO Welcome to the CDS
2023-03-15 06:11:20,348 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land
2023-03-15 06:1

2023-03-15 06:12:03,506 INFO Download rate 10.8M/s                                                                                                  
2023-03-15 06:12:03,581 INFO Welcome to the CDS
2023-03-15 06:12:03,582 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land
2023-03-15 06:12:03,705 INFO Request is queued
2023-03-15 06:12:04,762 INFO Request is running
2023-03-15 06:12:36,255 INFO Request is completed
2023-03-15 06:12:36,256 INFO Downloading https://download-0005-clone.copernicus-climate.eu/cache-compute-0005/cache/data1/adaptor.mars.internal-1678857146.5025694-20257-10-b13f2934-88ee-498f-8146-9bb5fdd4076c.nc to /Users/camillemodeste/Documents/Workflow/Pyronear/pyro-risks/pyro_risks/notebooks/datastore/ERA5Land/2011/era5land_2011_02_05.nc (4.5M)
2023-03-15 06:12:37,304 INFO Download rate 4.3M/s                                                                                                   
2023-03-15 06:12:37,383 INFO Welcome t

2023-03-15 06:14:57,784 INFO Download rate 9M/s                                                                                                     
2023-03-15 06:14:57,866 INFO Welcome to the CDS
2023-03-15 06:14:57,867 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land
2023-03-15 06:14:58,021 INFO Downloading https://download-0018.copernicus-climate.eu/cache-compute-0018/cache/data8/adaptor.mars.internal-1678731187.4406164-13937-20-edfd08d1-1506-44b6-80a4-93ea8b70118a.nc to /Users/camillemodeste/Documents/Workflow/Pyronear/pyro-risks/pyro_risks/notebooks/datastore/ERA5Land/2011/era5land_2011_02_16.nc (4.5M)
2023-03-15 06:14:58,613 INFO Download rate 7.6M/s                                                                                                   
2023-03-15 06:14:58,706 INFO Welcome to the CDS
2023-03-15 06:14:58,707 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land
2023-03-15 06:14:58,8

...

Unexpected error: <class 'Exception'>


2023-03-18 12:27:49,198 INFO Request is running
2023-03-18 12:28:20,690 INFO Request is completed
2023-03-18 12:28:20,691 INFO Downloading https://download-0015-clone.copernicus-climate.eu/cache-compute-0015/cache/data2/adaptor.mars.internal-1679138890.2051833-11041-12-e7686352-d56e-4638-83f4-31225375eb36.nc to /Users/camillemodeste/Documents/Workflow/Pyronear/pyro-risks/pyro_risks/notebooks/datastore/ERA5Land/2020/era5land_2020_12_01.nc (4.5M)
2023-03-18 12:28:21,659 INFO Download rate 4.6M/s                                                                                                   
2023-03-18 12:28:21,750 INFO Welcome to the CDS
2023-03-18 12:28:21,752 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land
2023-03-18 12:28:21,803 INFO Request is queued
2023-03-18 12:28:22,860 INFO Request is running
2023-03-18 12:28:54,384 INFO Request is completed
2023-03-18 12:28:54,386 INFO Downloading https://download-0012-clone.copernicus-climate.e

2023-03-18 12:33:58,855 INFO Downloading https://download-0019.copernicus-climate.eu/cache-compute-0019/cache/data5/adaptor.mars.internal-1679139228.1033802-719-11-47bc1694-1e46-4cc2-82d3-a3d7770c7be8.nc to /Users/camillemodeste/Documents/Workflow/Pyronear/pyro-risks/pyro_risks/notebooks/datastore/ERA5Land/2020/era5land_2020_12_11.nc (4.5M)
2023-03-18 12:34:00,461 INFO Download rate 2.8M/s                                                                                                   
2023-03-18 12:34:00,562 INFO Welcome to the CDS
2023-03-18 12:34:00,564 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land
2023-03-18 12:34:00,623 INFO Request is queued
2023-03-18 12:34:01,685 INFO Request is running
2023-03-18 12:34:33,245 INFO Request is completed
2023-03-18 12:34:33,247 INFO Downloading https://download-0006-clone.copernicus-climate.eu/cache-compute-0006/cache/data5/adaptor.mars.internal-1679139262.264111-29001-5-0f93b9bb-8c20-4379-8c2c-6

2023-03-18 12:39:38,531 INFO Download rate 4.4M/s                                                                                                   
2023-03-18 12:39:38,623 INFO Welcome to the CDS
2023-03-18 12:39:38,624 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land
2023-03-18 12:39:38,694 INFO Request is queued
2023-03-18 12:39:39,765 INFO Request is running
2023-03-18 12:40:11,316 INFO Request is completed
2023-03-18 12:40:11,319 INFO Downloading https://download-0005-clone.copernicus-climate.eu/cache-compute-0005/cache/data6/adaptor.mars.internal-1679139601.7706335-1437-5-08280d53-d9f7-47a2-aef8-7449fee8f2a7.nc to /Users/camillemodeste/Documents/Workflow/Pyronear/pyro-risks/pyro_risks/notebooks/datastore/ERA5Land/2020/era5land_2020_12_22.nc (4.5M)
2023-03-18 12:40:12,675 INFO Download rate 3.3M/s                                                                                                   
2023-03-18 12:40:12,778 INFO Welcome to 

2023-03-18 12:45:21,900 INFO Download rate 7.2M/s                                                                                                   


# 2011_08_27

In [7]:
"""year=2022
month=6
day=30
datastore_path = os.getcwd()+"/pyro_risks/notebooks/datastore"
era5land_path = os.path.join(datastore_path, f"era5land_{year}_{month}_{day}.nc")"""

'year=2022\nmonth=6\nday=30\ndatastore_path = os.getcwd()+"/pyro_risks/notebooks/datastore"\nera5land_path = os.path.join(datastore_path, f"era5land_{year}_{month}_{day}.nc")'

#### Exploration du dataset et transformation des données

In [35]:
def relative_humidity_calc(d2m, t2m):
    e = (17.1*d2m)/(235+d2m)
    es = (17.1*t2m)/(235+t2m)
    return np.exp(e)/np.exp(es)

In [49]:
era5_dir = os.getcwd()+"/pyro_risks/notebooks/datastore/ERA5Land"
for folder in os.listdir(era5_dir):
    print(era5_dir+"/"+folder)

/Users/camillemodeste/Documents/Workflow/Pyronear/pyro-risks/pyro_risks/notebooks/datastore/ERA5Land/2013
/Users/camillemodeste/Documents/Workflow/Pyronear/pyro-risks/pyro_risks/notebooks/datastore/ERA5Land/2014
/Users/camillemodeste/Documents/Workflow/Pyronear/pyro-risks/pyro_risks/notebooks/datastore/ERA5Land/2022
/Users/camillemodeste/Documents/Workflow/Pyronear/pyro-risks/pyro_risks/notebooks/datastore/ERA5Land/2015
/Users/camillemodeste/Documents/Workflow/Pyronear/pyro-risks/pyro_risks/notebooks/datastore/ERA5Land/2012
/Users/camillemodeste/Documents/Workflow/Pyronear/pyro-risks/pyro_risks/notebooks/datastore/ERA5Land/.DS_Store
/Users/camillemodeste/Documents/Workflow/Pyronear/pyro-risks/pyro_risks/notebooks/datastore/ERA5Land/2020.csv
/Users/camillemodeste/Documents/Workflow/Pyronear/pyro-risks/pyro_risks/notebooks/datastore/ERA5Land/2021.csv
/Users/camillemodeste/Documents/Workflow/Pyronear/pyro-risks/pyro_risks/notebooks/datastore/ERA5Land/2009.csv
/Users/camillemodeste/Documen

In [46]:
era5_dir = os.getcwd()+"/pyro_risks/notebooks/datastore/ERA5Land"
for folder in os.listdir(era5_dir):
    era5_dir_folder = era5_dir+"/"+folder
    if not folder.__contains__('.'):
        df_year = pd.DataFrame()
        for file in os.listdir(era5_dir_folder):
            file_path = era5_dir_folder+"/"+file
            ds = xr.open_dataset(file_path)
            data = ds.to_dataframe()

            # Drop NaNs (nulle part)
            data = data.dropna()
            data = data.reset_index()

            data["time"] = pd.to_datetime(
                data["time"], format="%Y-%m-%d %H:%M:%S", errors="coerce"
            )
            data["time"] = data["time"].dt.normalize()

            # Wind speed
            data["wind_speed"] = np.sqrt(data["u10"].multiply(data["u10"], axis="index")+data["v10"].multiply(data["v10"], axis="index"))

            # Relative humidity
            data["relative_humidity"] = data.apply(lambda row: relative_humidity_calc(row['d2m'], row['t2m']), axis=1)

            # Aggrégation pour avoir les données max d'un point sur 24h
            data_aggregation_max = data.drop(columns=['tp'], inplace=False)
            data_aggregation_max = data_aggregation_max.groupby(['longitude', 'latitude','time'], as_index=False).max()
            new_col_names=['longitude', 'latitude','date']
            for col_name in data_aggregation_max.columns[3:]:
                new_col_name = col_name+"_max"
                new_col_names.append(new_col_name)
            data_aggregation_max.columns = new_col_names

            # Aggrégation pour avoir les données totales d'un point sur 24h
            data_aggregation_sum = data.drop(columns=['time','t2m','d2m','sp','u10','v10','wind_speed','relative_humidity'], inplace=False)
            data_aggregation_sum = data_aggregation_sum.groupby(['longitude', 'latitude'], as_index=False).sum(numeric_only=True)
            new_col_names=['longitude', 'latitude']
            for col_name in data_aggregation_sum.columns[2:]:
                new_col_name = col_name+"_sum"
                new_col_names.append(new_col_name)
            data_aggregation_sum.columns = new_col_names

            ## Aggrégation pour avoir les données min d'un point sur 24h
            data_aggregation_min = data.drop(columns=['time','t2m','d2m','sp','u10','v10','wind_speed','tp'], inplace=False)
            data_aggregation_min = data_aggregation_min.groupby(['longitude', 'latitude'], as_index=False).min(numeric_only=True)
            new_col_names=['longitude', 'latitude']
            for col_name in data_aggregation_min.columns[2:]:
                new_col_name = col_name+"_min"
                new_col_names.append(new_col_name)
            data_aggregation_min.columns = new_col_names


            # On reconsolide
            data_aggregated = pd.merge(data_aggregation_max, data_aggregation_sum,  how='left', on=['longitude', 'latitude'])
            data_aggregated = pd.merge(data_aggregated, data_aggregation_min,  how='left', on=['longitude', 'latitude'])

            # On ajoute la donnée au dataset de l'année
            df_year= pd.concat([df_year, data_aggregated])
        # On sauvegarde la data en csv
        csv_path = era5_dir+"/"+folder+".csv"
        df_year.to_csv(csv_path, index=False)

In [41]:
"""ds = xr.open_dataset(file_path)
data = ds.to_dataframe()

# Drop NaNs (nulle part)
data = data.dropna()
data = data.reset_index()

data["time"] = pd.to_datetime(
    data["time"], format="%Y-%m-%d %H:%M:%S", errors="coerce"
)
data["time"] = data["time"].dt.normalize()

# Wind speed
data["wind_speed"] = np.sqrt(data["u10"].multiply(data["u10"], axis="index")+data["v10"].multiply(data["v10"], axis="index"))

# Relative humidity
data["relative_humidity"] = data.apply(lambda row: relative_humidity_calc(row['d2m'], row['t2m']), axis=1)"""

'ds = xr.open_dataset(file_path)\ndata = ds.to_dataframe()\n\n# Drop NaNs (nulle part)\ndata = data.dropna()\ndata = data.reset_index()\n\ndata["time"] = pd.to_datetime(\n    data["time"], format="%Y-%m-%d %H:%M:%S", errors="coerce"\n)\ndata["time"] = data["time"].dt.normalize()\n\n# Wind speed\ndata["wind_speed"] = np.sqrt(data["u10"].multiply(data["u10"], axis="index")+data["v10"].multiply(data["v10"], axis="index"))\n\n# Relative humidity\ndata["relative_humidity"] = data.apply(lambda row: relative_humidity_calc(row[\'d2m\'], row[\'t2m\']), axis=1)'

In [51]:
# Visualisation de la donnée à ce stade
"""data"""

,longitude,latitude,time,t2m,d2m,sp,u10,v10,tp,wind_speed,relative_humidity
0,-6.0,43.500000,2022-06-30,286.183228,283.670197,99259.468750,2.903868,-0.511924,7.153798e-03,2.948647,0.963331
1,-6.0,43.500000,2022-06-30,286.064209,283.755310,99270.093750,3.034442,-0.502382,7.120892e-05,3.075748,0.966257
2,-6.0,43.500000,2022-06-30,285.970886,283.737823,99218.812500,2.998321,-0.371521,1.143087e-04,3.021251,0.967340
3,-6.0,43.500000,2022-06-30,285.705414,283.532654,99194.320312,3.084473,-0.262697,1.911409e-04,3.095639,0.968180
4,-6.0,43.500000,2022-06-30,285.329498,283.040253,99199.406250,3.168381,-0.058454,2.745315e-04,3.168920,0.966448
...,...,...,...,...,...,...,...,...,...,...,...
265363,10.0,44.099998,2022-06-30,297.038483,285.755951,99112.085938,0.529986,0.086492,-1.862645e-09,0.536997,0.849046
265364,10.0,44.099998,2022-06-30,296.624878,285.789764,99119.015625,-0.654151,-0.156600,-1.862645e-09,0.672634,0.854478
265365,10.0,44.099998,2022-06-30,296.460022,285.906738,99168.453125,-0.355985,-0.066860,-1.862645e-09,0.362210,0.857970
265366,10.0,44.099998,2022-06-30,296.157318,286.291473,99179.539062,-0.303262,-0.563269,-1.862645e-09,0.639718,0.866595


Je regarde si j'ai bien 24x chaque point (utile pour + tard) :

In [52]:
"""data.groupby(['longitude', 'latitude']).size().max()"""

24

C'est bien le cas !

Il s'agit désormais d'aggréger nos données. On les aggrège par point, car chaque point apparaît ici 24x pour les 24h de la journée.

Il est stipulé dans le papier que l'on veut :
 - maximum 2 m temperature => aggrégation "max"
 - maximum wind speed => aggrégation "max"
 - maximum 2 m dewpoint temperature => aggrégation "max"
 - maximum surface pressure => aggrégation "max"
 - minimum relative humidity (donnée absente, à calculer, selon la doc "dewpoint temperature combined with temperature and pressure, it can be used to calculate the relative humidity", on va utiliser ce qui est écrit ici : https://earthscience.stackexchange.com/questions/24156/era5-single-level-calculate-relative-humidity, semble en accord avec la manière de calculer de ERA5 https://www.ecmwf.int/sites/default/files/elibrary/2016/17117-part-iv-physical-processes.pdf#subsection.7.4.2) => test sur plusieurs jours, semble ok
 - total precipitation => aggrégation "sum"

Donc on va devoir faire de multiples aggrégations, et d'autres transformations de données ...

In [53]:
"""# Aggrégation pour avoir les données max d'un point sur 24h
data_aggregation_max = data.drop(columns=['tp'], inplace=False)
data_aggregation_max = data_aggregation_max.groupby(['longitude', 'latitude','time'], as_index=False).max()
new_col_names=['longitude', 'latitude','date']
for col_name in data_aggregation_max.columns[3:]:
    new_col_name = col_name+"_max"
    new_col_names.append(new_col_name)
data_aggregation_max.columns = new_col_names

# Aggrégation pour avoir les données totales d'un point sur 24h
data_aggregation_sum = data.drop(columns=['time','t2m','d2m','sp','u10','v10','wind_speed','relative_humidity'], inplace=False)
data_aggregation_sum = data_aggregation_sum.groupby(['longitude', 'latitude'], as_index=False).sum(numeric_only=True)
new_col_names=['longitude', 'latitude']
for col_name in data_aggregation_sum.columns[2:]:
    new_col_name = col_name+"_sum"
    new_col_names.append(new_col_name)
data_aggregation_sum.columns = new_col_names

## Aggrégation pour avoir les données min d'un point sur 24h
data_aggregation_min = data.drop(columns=['time','t2m','d2m','sp','u10','v10','wind_speed','tp'], inplace=False)
data_aggregation_min = data_aggregation_min.groupby(['longitude', 'latitude'], as_index=False).min(numeric_only=True)
new_col_names=['longitude', 'latitude']
for col_name in data_aggregation_min.columns[2:]:
    new_col_name = col_name+"_min"
    new_col_names.append(new_col_name)
data_aggregation_min.columns = new_col_names


# On reconsolide
data_aggregated = pd.merge(data_aggregation_max, data_aggregation_sum,  how='left', on=['longitude', 'latitude'])
data_aggregated = pd.merge(data_aggregated, data_aggregation_min,  how='left', on=['longitude', 'latitude'])"""

In [54]:
data_aggregated

,longitude,latitude,date,t2m_max,d2m_max,sp_max,u10_max,v10_max,wind_speed_max,relative_humidity_max,tp_sum,relative_humidity_min
0,-6.0,41.000000,2022-06-30,296.987518,281.278259,92811.515625,2.454937,-1.533593,3.124889,0.936056,8.922070e-07,0.730093
1,-6.0,41.099998,2022-06-30,296.979797,281.002716,92889.593750,2.571152,-1.405686,3.080780,0.935651,8.922070e-07,0.730203
2,-6.0,41.200001,2022-06-30,296.788422,280.861267,92468.234375,2.644291,-1.260057,3.001338,0.935279,8.922070e-07,0.728778
3,-6.0,41.299999,2022-06-30,296.559784,280.646362,92195.640625,2.640478,-0.989248,2.931790,0.934290,8.922070e-07,0.730786
4,-6.0,41.400002,2022-06-30,296.966125,280.687531,92848.937500,2.520224,-0.632788,3.183845,0.931079,8.922070e-07,0.725903
...,...,...,...,...,...,...,...,...,...,...,...,...
11052,10.0,50.599998,2022-06-30,299.551788,289.785553,94953.437500,2.117734,1.220166,2.444096,0.992582,4.433718e-03,0.811276
11053,10.0,50.700001,2022-06-30,300.104553,290.316803,96087.703125,1.994789,1.103163,2.279507,0.991403,3.113521e-03,0.813633
11054,10.0,50.799999,2022-06-30,300.409393,290.703491,96983.554688,1.760339,0.790097,1.929520,0.991819,2.913952e-03,0.818797
11055,10.0,50.900002,2022-06-30,300.360596,290.656860,97532.898438,1.544512,0.773966,1.730040,0.992358,2.883021e-03,0.828902


#### Transformation en dataframe geopandas

In [56]:
geo_data_aggregated = gpd.GeoDataFrame(
    data_aggregated,
    geometry=gpd.points_from_xy(data_aggregated["longitude"], data_aggregated["latitude"]),
    crs="EPSG:4326",
)
geo_data_aggregated = geo_data_aggregated.drop(['longitude', 'latitude'], axis=1)

In [57]:
# Visualisation de la donnée à ce stade
geo_data_aggregated

,date,t2m_max,d2m_max,sp_max,u10_max,v10_max,wind_speed_max,relative_humidity_max,tp_sum,relative_humidity_min,geometry
0,2022-06-30,296.987518,281.278259,92811.515625,2.454937,-1.533593,3.124889,0.936056,8.922070e-07,0.730093,POINT (-6.00000 41.00000)
1,2022-06-30,296.979797,281.002716,92889.593750,2.571152,-1.405686,3.080780,0.935651,8.922070e-07,0.730203,POINT (-6.00000 41.10000)
2,2022-06-30,296.788422,280.861267,92468.234375,2.644291,-1.260057,3.001338,0.935279,8.922070e-07,0.728778,POINT (-6.00000 41.20000)
3,2022-06-30,296.559784,280.646362,92195.640625,2.640478,-0.989248,2.931790,0.934290,8.922070e-07,0.730786,POINT (-6.00000 41.30000)
4,2022-06-30,296.966125,280.687531,92848.937500,2.520224,-0.632788,3.183845,0.931079,8.922070e-07,0.725903,POINT (-6.00000 41.40000)
...,...,...,...,...,...,...,...,...,...,...,...
11052,2022-06-30,299.551788,289.785553,94953.437500,2.117734,1.220166,2.444096,0.992582,4.433718e-03,0.811276,POINT (10.00000 50.60000)
11053,2022-06-30,300.104553,290.316803,96087.703125,1.994789,1.103163,2.279507,0.991403,3.113521e-03,0.813633,POINT (10.00000 50.70000)
11054,2022-06-30,300.409393,290.703491,96983.554688,1.760339,0.790097,1.929520,0.991819,2.913952e-03,0.818797,POINT (10.00000 50.80000)
11055,2022-06-30,300.360596,290.656860,97532.898438,1.544512,0.773966,1.730040,0.992358,2.883021e-03,0.828902,POINT (10.00000 50.90000)


#### Avantages/Inconvénients de ce dataset : 

Avantages / Inconvénients :
 - difficile à requêter, je n'ai étudié que 1 jour car au-delà l'API soulevait une erreur (trop de données requêtées)
 
 
 => Solution ici = télécharger le dataset en dur sur un drive, si on veut jouer avec plein de données => mais même en téléchargeant le dataset sur ERA5, on est limités 

### 2. Satellite variables - MODIS

MODIS NDVI : dataset MOD13Q1.061 : tous les 16 jours
MODIS LST : dataset MOD11C1.061 : daily

In [50]:
data_folder_path = "pyro_risks/notebooks/datastore/NASA Satellite/"
modis_ndvi = "MOD13C1.061.ncml.csv"

df_modis_ndvi = pd.read_csv(data_folder_path+modis_ndvi,sep=",")
df_modis_ndvi

ParserError: Error tokenizing data. C error: Expected 3601 fields in line 3, saw 7201


### 3. Soil moisture index - European Drought Observatory

On le télécharge sur le site suivant : https://edo.jrc.ec.europa.eu/gdo/php/index.php?id=2112

Documentation : https://edo.jrc.ec.europa.eu/documents/factsheets/factsheet_soilmoisture.pdf

In [58]:
smi_filepath = "pyro_risks/data/edo_soil_moisture_index_20230101_20230211_t.nc"

smi_ds = xr.open_dataset(smi_filepath)
smi_data = smi_ds.to_dataframe()

In [59]:
smi_data

sminx          3035
time       lat     lon                         
2023-01-01 5497500 2502500    NaN -2.147484e+09
                   2507500    NaN -2.147484e+09
                   2512500    NaN -2.147484e+09
                   2517500    NaN -2.147484e+09
                   2522500    NaN -2.147484e+09
...                           ...           ...
2023-02-11 752500  7477500    NaN -2.147484e+09
                   7482500    NaN -2.147484e+09
                   7487500    NaN -2.147484e+09
                   7492500    NaN -2.147484e+09
                   7497500    NaN -2.147484e+09

[4750000 rows x 2 columns]

### 4. Roads distance, waterway distance, and yearly population density - WorldPop

### 5. Elevation and Slope - Copernicus EU-DEM

### 6. Ten variables with the fraction of classes - Copernicus Corine Land Cover